Setting virtual environment:
python -m venv env
.\env\Scripts\activate
base model
Importing Packages:
pip install -U langchain-ollama
pip install langchain
pip install crewai
pip install --upgrade langchain langchain_community
pip install pypdf
pip install faiss-cpu
pip install duckduckgo-search
pip install langchain-groq
pip install streamlit
pip install -U sentence-transformers
pip install langchain_huggingface
running the streamlit code: python -m streamlit run app.py

In [1]:
import langchain
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
llama_llm = OllamaLLM(model="llama3")

In [4]:
parser = StrOutputParser()
llama_chain = llama_llm | parser

In [5]:
file_loader = PyPDFLoader("Cricket Rules.pdf")
page = file_loader.load_and_split()

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
chunks = splitter.split_documents(page)

In [7]:
embeddings = OllamaEmbeddings(
    model="llama3", )
vector_database = FAISS.from_documents(chunks, embeddings) 
retriever = vector_database.as_retriever()

In [8]:
example_template = ("""
You are a AI powered chatbot designed to provide information and knowledge 
regarding the cricket sport based only on the context provided to you.
Context = {context}
Question = {question}
""")

In [9]:
prompt = PromptTemplate.from_template(template=example_template)
print(prompt.format(
    context = "This is the provided context",
    question = "Answer this question"
))


You are a AI powered chatbot designed to provide information and knowledge 
regarding the cricket sport based only on the context provided to you.
Context = This is the provided context
Question = Answer this question



In [10]:
result = RunnableParallel(context = retriever, question = RunnablePassthrough())

In [11]:
final_chain = result | prompt | llama_llm | parser

In [12]:
search = DuckDuckGoSearchRun(proxies=None, verify_ssl=True)

In [13]:
load_dotenv()

True

In [14]:
groq_llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"), temperature=0.3)

In [15]:
document_chain = create_stuff_documents_chain(llama_llm,prompt, document_variable_name="context",
output_parser=parser)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [38]:
def get_answer(question):
    try:
        # Get relevant documents and their similarity scores
        context = retriever.get_relevant_documents(question)
        print(f"Found {len(context)} context documents")
        
        # Check if the context is actually relevant
        question_words = set(question.lower().split())
        context_text = " ".join([doc.page_content.lower() for doc in context])
        
        matching_words = sum(1 for word in question_words if word in context_text)
        is_relevant = matching_words >= 2
        
        if not is_relevant:
            print("Context not relevant, using DuckDuckGo search...")
            search_result = search.run(question)
            return f"""
            DuckDuckGo Search Result:
            {search_result}
            """
        
        print("Using PDF context for answer...")
        context_text = "\n".join([doc.page_content for doc in context])
        
        groq_prompt = f"""
        Use the following context from cricket rules to answer the question:
        
        Context: {context_text}
        
        Question: {question}
        
        If the context doesn't contain relevant information to answer the question, 
        please explicitly state that.
        """
        
        groq_answer = groq_llm.invoke(groq_prompt)
        
        # If Groq indicates no relevant information, fallback to DuckDuckGo
        if any(phrase in groq_answer.content.lower() for phrase in 
              ["does not contain", "no information", "cannot answer", "don't have"]):
            print("Groq indicated no relevant information, falling back to DuckDuckGo...")
            search_result = search.run(question)
            return f"""
            DuckDuckGo Search Result (Fallback):
            {search_result}
            """
        
        return f"""
        Groq API Response (Based on PDF context):
        {groq_answer.content}
        """
            
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        search_result = search.run(question)
        return f"""
        DuckDuckGo Search Result (Error Fallback):
        {search_result}
        """

In [45]:
question = "what is the bowl used in cricket for?"
print(get_answer(question))

Found 4 context documents
Using PDF context for answer...

        Groq API Response (Based on PDF context):
        The bowl used in cricket is used by the bowler to deliver the ball to the batsman. The bowler aims to get the batsman out by either hitting the wickets directly with the ball, having the batsman hit the ball in the air and catching it before it touches the ground, or by having the batsman hit the ball onto the wickets and dislodging the bails. The bowl is thrown in a specific way, using a particular action and following certain rules, in order to make it challenging for the batsman to hit the ball.
        


In [46]:
conversation_memory = ConversationBufferWindowMemory(
    k = 5,
    memory_key="chat_history",
    return_messages=True
)

def chat():
    print("\n=== Cricket Rules Chatbot ===")
    print("Special commands:")
    print("- 'quit': Exit the chat")
    print("- 'history': Show chat history")
    print("-" * 50)
    
    while True:
        # Get user input
        user_input = input("You: ")
        print(f"\nQuestion: {user_input}")
        print("-" * 50)
        
        # Check for special commands
        if user_input.lower() == 'quit':
            print("\nGoodbye!")
            break
        elif user_input.lower() == 'history':
            # Get messages from memory
            chat_history = conversation_memory.load_memory_variables({})["chat_history"]
            print("\nChat History:")
            if not chat_history:
                print("No chat history yet!")
            else:
                print(chat_history)
            print("-" * 50)
            continue
        
        # Get and display response
        try:
            response = get_answer(user_input)
            print("\nAssistant:")
            print(response)
            print("-" * 50)
            
            # Save to memory (using the correct format)
            conversation_memory.save_context(
                {"input": user_input},
                {"output": response}
            )
            
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("-" * 50)

# Run the chatbot
chat()


=== Cricket Rules Chatbot ===
Special commands:
- 'quit': Exit the chat
- 'history': Show chat history
--------------------------------------------------

Question: what is the bat used in cricket for?
--------------------------------------------------
Found 4 context documents
Using PDF context for answer...

Assistant:

        Groq API Response (Based on PDF context):
        The context provided does not give information about the specific use of the bat in cricket, but it does mention several players and their achievements in the sport. The bat is one of the essential pieces of equipment in cricket, used by the batsman to hit the ball and score runs. If you need more detailed information about how the bat is used in cricket, I would recommend looking up the rules and gameplay explanations of cricket.
        
--------------------------------------------------

Question: 
--------------------------------------------------
Found 4 context documents
Context not relevant, using DuckD